# Lecture 7: Convolutional networks

Notebook adapted from [Deep Learning (with PyTorch)](https://github.com/Atcold/pytorch-Deep-Learning) by Alfredo Canziani. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# MNIST

In [ ]:
tf = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=True, transform=tf),
                                           batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=tf),
                                          batch_size=1000, shuffle=True)

In [ ]:
batch = next(iter(train_loader))
x = batch[0][:10]
y = batch[1][:10]

fig, axs = plt.subplots(1, 5, figsize=(12, 4))

for i in range(5):
    axs[i].imshow(x[i].squeeze().numpy())

# MLP vs ConvNet

In [ ]:
class MLP(nn.Module):
    def __init__(self, D, H, C):
        super().__init__()
        self.D = D
        self.net = nn.Sequential(
            nn.Linear(D, H),
            nn.ReLU(),
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Linear(H, C),
            # nn.Softmax(dim=-1)
        )
        
    def forward(self, x):
        x = x.view(-1, self.D)
        return self.net(x)

class ConvNet(nn.Module):
    def __init__(self, D, n_kernels, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=n_kernels, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=n_kernels, out_channels=n_kernels, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(n_kernels * 4 * 4, 50),
            nn.ReLU(),
            nn.Linear(50, C),
            # nn.Softmax(dim=-1)
        )
        
    def forward(self, x):
        return self.net(x)

In [ ]:
def train(model, perm=torch.arange(0, 784).long(), n_epochs=1):
    model.train()    
    optimizer = torch.optim.AdamW(model.parameters())
    
    for epoch in range(n_epochs):
        for i, (data, target) in enumerate(train_loader):
            # send to device
            data, targets = data.to(device), target.to(device)

            # permute pixels
            data = data.view(-1, 28*28)
            data = data[:, perm]
            data = data.view(-1, 1, 28, 28)

            # step
            optimizer.zero_grad()
            logits = model(data)
            
            loss = F.cross_entropy(logits, targets)
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print(f"epoch={epoch}, step={i}: train loss={loss.item():.4f}")

In [ ]:
def test(model, perm=torch.arange(0, 784).long()):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    for data, targets in test_loader:
        # send to device
        data, targets = data.to(device), targets.to(device)
        
        # permute pixels
        data = data.view(-1, 28*28)
        data = data[:, perm]
        data = data.view(-1, 1, 28, 28)
        
        # metrics
        logits = model(data)
        test_loss += F.cross_entropy(logits, targets, reduction='sum').item()
        preds = torch.argmax(logits, dim=1)     
        correct += (preds == targets).sum()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    
    print(f"test loss={test_loss:.4f}, accuracy={accuracy:.4f}")

In [ ]:
# MLP
D = 28*28  
C = 10   
H = 8

mlp = MLP(D, H, C)
mlp.to(device)
print(f"Parameters={sum(p.numel() for p in mlp.parameters())/1e3}K")

train(mlp)
test(mlp)

In [ ]:
# ConvNet, with the same number of parameters
n_kernels = 6

convnet = ConvNet(D, n_kernels, C)
convnet.to(device)
print(f"Parameters={sum(p.numel() for p in convnet.parameters())/1e3}K")

train(convnet)
test(convnet)

The convolutional network performs better with the same number of parameters, thanks to its use of prior knowledge about images:

* Use of convolution: Locality and stationarity in images
* Pooling: builds in some translation invariance

What if those assumptions are wrong?

# MLP vs ConvNet, on shuffled pixels

In [ ]:
perm = torch.randperm(784)

batch = next(iter(train_loader))
x = batch[0][:10]
y = batch[1][:10]

fig, axs = plt.subplots(1, 5, figsize=(12, 4))

for i in range(5):
    axs[i].imshow(x[i].squeeze().numpy())
    
fig, axs = plt.subplots(1, 5, figsize=(12, 4))
x = x.view(-1, 28*28)
x = x[:, perm]
x = x.view(-1, 1, 28, 28)

for i in range(5):
    axs[i].imshow(x[i].squeeze().numpy())

In [ ]:
# ConvNet on shuffled pixels
n_kernels = 6

convnet = ConvNet(D, n_kernels, C)
convnet.to(device)
print(f"Parameters={sum(p.numel() for p in convnet.parameters())/1e3}K")

train(convnet, perm=perm)
test(convnet, perm=perm)

In [ ]:
# MLP on shuffled pixels
D = 28*28  
C = 10   
H = 8

mlp = MLP(D, n_hidden, output_size)
mlp.to(device)
print(f"Parameters={sum(p.numel() for p in mlp.parameters())/1e3}K")

train(mlp, perm=perm)
test(mlp, perm=perm)

The convolutional network's performance drops when we permute the pixels, but the MLP's performance stays the same.
* ConvNet makes the assumption that pixels lie on a grid and are stationary/local.
* It loses performance when this assumption is wrong.
* The fully-connected network does not make this assumption.
* It does less well when it is true, since it doesn't take advantage of this prior knowledge.
* But it doesn't suffer when the assumption is wrong.